### Config

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from chronos_mlx import ChronosPipeline
pd.set_option('display.max_columns', None)

In [2]:
year = pd.read_parquet('/Users/tomaltenborg/Documents/Master/Master thesis/Notebooks/M3 Data/M3_yearly_processed.parquet')

## Loop and Produce Forecasts for All Yearly Series

In [5]:
pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-base",
    dtype="bfloat16",
)

unique_series = year['Series'].unique()

forecasts = {}
counter = 0
for series_id in unique_series:
    # Filter the dataframe for the current series
    series_data = year[year['Series'] == series_id]
    
    # Ensure the data is sorted by date
    series_data = series_data.sort_values('Date')
    
    # Get the number of observations to use for forecasting
    N = series_data['N'].iloc[0]
    NF = series_data['NF'].iloc[0]
    
    # Get the date of the last observation used for training
    last_training_date = series_data['Date'].iloc[N-NF-1]  # -1 because iloc is 0-based

    # Prepare the input data for the model
    training_data = series_data['Value'].iloc[:N-NF].values  # Exclude the last NF values
    
    # Calculate forecast dates assuming yearly data
    forecast_dates = pd.date_range(start=last_training_date, periods=NF+1, freq='YS')[1:]  # Skip the first since it's the last training date
    
    # Predict the next NF values
    try:
        model_forecasts = pipeline.predict(
            context=training_data,
            prediction_length=NF,
            num_samples=N-NF
        )#.squeeze(0).squeeze(0).numpy()  # Remove batch and sample dimensions for simplicity
    except Exception as e:
        print(f"Failed to predict for series {series_id} with error: {e}")
        continue
    counter += 1
    if counter > 100:
        print('100 series forecasted')
        counter = 0
    # Quantiles for uncertainty bounds
    low, median, high = np.quantile(model_forecasts[0], [0.1, 0.5, 0.9], axis=0)
    forecasts[series_id] = {
        'Date': forecast_dates,
        'low': low,
        'median': median,
        'high': high
    }

# Convert forecasts dictionary to a DataFrame for easier manipulation
forecast_df = pd.DataFrame()
for series_id, data in forecasts.items():
    df_temp = pd.DataFrame({
        'Date': data['Date'],
        'Series_ID': series_id,
        'Low': data['low'],
        'Median': data['median'],
        'High': data['high']
    })
    forecast_df = pd.concat([forecast_df, df_temp], ignore_index=True)

# forecast_df.set_index(['Date', 'Series_ID'], inplace=True)
print('Forecasting completed for all series.')

100 series forecasted
100 series forecasted
100 series forecasted
100 series forecasted
100 series forecasted
100 series forecasted
Forecasting completed for all series.


In [6]:
forecast_df.loc[forecast_df['Series_ID'] == 1]

,Date,Series_ID,Low,Median,High
0,1989-01-01,1,5232.904222,5405.892792,5804.518625
1,1990-01-01,1,5445.379313,5828.962662,6287.758433
2,1991-01-01,1,5298.715091,6186.221664,6686.384267
3,1992-01-01,1,5150.170559,6139.213901,7163.983143
4,1993-01-01,1,4751.544725,5998.190611,7489.276866
5,1994-01-01,1,4603.000193,5603.325398,8060.891269


In [7]:
forecast_df.to_parquet('M3_Chronos_year_forecast.parquet', index=False,  compression='gzip', engine='pyarrow')